In [ ]:
import json
import base64
import requests
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage, SystemMessage
from typing import Dict, List, cast, Optional, Any
from langchain_core.tools import tool
from langchain.agents import AgentExecutor
from langchain_core.prompts import PromptTemplate

from dataclasses import dataclass, field
from typing import Annotated, Sequence
from langgraph.graph import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")

@dataclass
class State:
    session_id: str = ""
    github_url: str = ""
    company_name: str = ""
    messages: Annotated[Sequence[AnyMessage], add_messages] = field(default_factory=list)

# GitHub API 헬퍼 함수들
def _build_github_headers() -> dict:
    """GitHub API 헤더 생성"""
    headers = {"Accept": "application/vnd.github.v3+json"}
    if GITHUB_TOKEN:
        headers["Authorization"] = f"Bearer {GITHUB_TOKEN}"
    return headers

def _github_api_call(endpoint: str, params: Optional[Dict] = None) -> Any:
    """GitHub API 호출 헬퍼 함수"""
    headers = _build_github_headers()
    try:
        r = requests.get(f"https://api.github.com{endpoint}", headers=headers, params=params, timeout=10)
        if r.status_code == 200:
            return r.json()
        try:
            err_json = r.json()
        except Exception:
            err_json = {"message": r.text}
        return {"error": f"GitHub API {r.status_code}", "detail": err_json}
    except Exception as e:
        return {"error": str(e)}


In [2]:
# LangChain Tool 정의

@tool
def get_user_info(username: str) -> Dict[str, Any]:
    """
    Get GitHub user profile information using GitHub API
    
    Args:
        username: GitHub username
    
    Returns:
        사용자 프로필 정보 딕셔너리 (이름, bio, 팔로워 수 등)
    """
    result = _github_api_call(f"/users/{username}")
    if isinstance(result, dict) and "error" not in result:
        # 필요한 필드만 추출하여 반환
        return {
            "login": result.get("login"),
            "name": result.get("name"),
            "bio": result.get("bio"),
            "company": result.get("company"),
            "location": result.get("location"),
            "email": result.get("email"),
            "followers": result.get("followers"),
            "following": result.get("following"),
            "public_repos": result.get("public_repos"),
            "created_at": result.get("created_at"),
        }
    return result

@tool
def list_user_repos(username: str, limit: int = 30, sort: str = "updated") -> List[Dict[str, Any]]:
    """
    List repositories for a GitHub user using GitHub API
    
    Args:
        username: GitHub username
        limit: Maximum number of repos to return (default: 30)
        sort: Sort by updated, created, pushed, or full_name (default: updated)
    
    Returns:
        레포지토리 리스트
    """
    repos = []
    page = 1
    
    while len(repos) < limit:
        data = _github_api_call(
            f"/users/{username}/repos",
            {"per_page": min(100, limit - len(repos)), "page": page, "sort": sort}
        )
        if isinstance(data, dict) and "error" in data:
            return data
        if not data:
            break
        
        # 필요한 필드만 추출
        for repo in data:
            repos.append({
                "name": repo.get("name"),
                "full_name": repo.get("full_name"),
                "description": repo.get("description"),
                "html_url": repo.get("html_url"),
                "language": repo.get("language"),
                "stargazers_count": repo.get("stargazers_count"),
                "forks_count": repo.get("forks_count"),
                "updated_at": repo.get("updated_at"),
                "topics": repo.get("topics", []),
            })
        
        if len(data) < 100:
            break
        page += 1
        if page > 5:
            break
    
    return repos[:limit]

@tool
def get_repo_details(owner: str, repo: str) -> Dict[str, Any]:
    """
    Get detailed information about a specific repository using GitHub API
    
    Args:
        owner: Repository owner username
        repo: Repository name
    
    Returns:
        레포지토리 상세 정보 딕셔너리
    """
    result = _github_api_call(f"/repos/{owner}/{repo}")
    if isinstance(result, dict) and "error" not in result:
        return {
            "name": result.get("name"),
            "full_name": result.get("full_name"),
            "description": result.get("description"),
            "html_url": result.get("html_url"),
            "language": result.get("language"),
            "stargazers_count": result.get("stargazers_count"),
            "forks_count": result.get("forks_count"),
            "watchers_count": result.get("watchers_count"),
            "open_issues_count": result.get("open_issues_count"),
            "topics": result.get("topics", []),
            "created_at": result.get("created_at"),
            "updated_at": result.get("updated_at"),
            "pushed_at": result.get("pushed_at"),
            "license": result.get("license", {}).get("name") if result.get("license") else None,
        }
    return result

@tool
def get_repo_readme(owner: str, repo: str) -> Dict[str, Any]:
    """
    Get README content of a repository using GitHub API
    
    Args:
        owner: Repository owner username
        repo: Repository name
    
    Returns:
        README 내용 (최대 5000자)
    """
    data = _github_api_call(f"/repos/{owner}/{repo}/readme")
    if isinstance(data, dict) and data.get("content"):
        try:
            content = base64.b64decode(data["content"]).decode("utf-8", errors="replace")
            return {"content": content[:5000], "name": data.get("name"), "path": data.get("path")}
        except Exception as e:
            return {"error": f"Failed to decode README: {e}"}
    return data

@tool
def get_repo_commits(owner: str, repo: str, limit: int = 5) -> List[Dict[str, Any]]:
    """
    Get recent commits from a repository using GitHub API
    
    Args:
        owner: Repository owner username
        repo: Repository name
        limit: Number of commits to return (default: 5)
    
    Returns:
        최근 커밋 리스트
    """
    data = _github_api_call(f"/repos/{owner}/{repo}/commits", {"per_page": limit})
    if isinstance(data, list):
        commits = []
        for commit in data:
            commits.append({
                "sha": commit.get("sha"),
                "message": commit.get("commit", {}).get("message"),
                "author": commit.get("commit", {}).get("author", {}).get("name"),
                "date": commit.get("commit", {}).get("author", {}).get("date"),
                "html_url": commit.get("html_url"),
            })
        return commits
    return data

@tool
def get_repo_languages(owner: str, repo: str) -> Dict[str, int]:
    """
    Get programming languages used in a repository using GitHub API
    
    Args:
        owner: Repository owner username
        repo: Repository name
    
    Returns:
        사용된 프로그래밍 언어와 바이트 수 딕셔너리
    """
    return _github_api_call(f"/repos/{owner}/{repo}/languages")

@tool
def get_repo_contributors(owner: str, repo: str, limit: int = 10) -> List[Dict[str, Any]]:
    """
    Get contributors of a repository using GitHub API
    
    Args:
        owner: Repository owner username
        repo: Repository name
        limit: Number of contributors to return (default: 10)
    
    Returns:
        기여자 리스트
    """
    data = _github_api_call(f"/repos/{owner}/{repo}/contributors", {"per_page": limit})
    if isinstance(data, list):
        contributors = []
        for contributor in data:
            contributors.append({
                "login": contributor.get("login"),
                "contributions": contributor.get("contributions"),
                "html_url": contributor.get("html_url"),
                "type": contributor.get("type"),
            })
        return contributors
    return data


In [3]:
# GitHub Agent 정의

async def github_agent_token(state: State) -> Dict[str, List[AIMessage]]:
    """
    Analyze GitHub profile and repository information using GitHub API
    """

    AGENT_MODEL = "gpt-4o-mini"

    model = ChatOpenAI(model=AGENT_MODEL, temperature=0, api_key=OPENAI_API_KEY)

    # URL에서 username 또는 owner/repo 추출
    username = state.github_url.rstrip('/').split('/')[-1] if 'github.com/' in state.github_url else state.github_url
    print("username: ", username)
    
    # GitHub API 기반 LangChain tools
    tools = [
        get_user_info,
        list_user_repos,
        get_repo_details,
        get_repo_readme,
        get_repo_commits,
        get_repo_languages,
        get_repo_contributors,
    ]

    system_message = f"""You're an expert at analyzing GitHub repositories.
    You use GitHub API tools to analyze GitHub profiles and repositories.
    
    username: {username}

    You are a GitHub repository analyst with access to GitHub API tools.
    Analyze the given GitHub profile/repositories based on the user's query.
    Use the provided tools as many times as needed to gather sufficient information.
    Always respond in Korean.
    
    Available GitHub API tools:
    - get_user_info: Get user profile information (followers, bio, etc.)
    - list_user_repos: List repositories for a user with sorting options
    - get_repo_details: Get detailed information about a specific repository
    - get_repo_readme: Get README content of a repository
    - get_repo_commits: Get recent commits from a repository
    - get_repo_languages: Get programming languages used in a repository
    - get_repo_contributors: Get contributors of a repository
    
    IMPORTANT INSTRUCTIONS:
    1. Use tools strategically to gather information (maximum 5-7 tool calls)
    2. GitHub API provides more accurate and structured data than web scraping
    3. After collecting sufficient information, provide a comprehensive Korean summary
    4. Do NOT continue using tools indefinitely - stop when you have enough information
    5. Your final response should be a complete analysis in Korean, not a request for more tools
    
    Process:
    1. Start with get_user_info or list_user_repos for basic info
    2. Use get_repo_details and get_repo_readme for specific repository analysis
    3. Use additional tools (commits, languages, contributors) if needed
    4. Provide final comprehensive summary in Korean and STOP
    """

    agent = create_react_agent(model, tools, prompt=system_message)

    # recursion_limit 설정으로 무한 루프 방지
    config = {"recursion_limit": 15, "max_iterations": 8}
    response = await agent.ainvoke({"messages": state.messages}, config=config)
    print("\n" + "="*60)
    print("Agent Response:")
    print("="*60)
    print(response["messages"][-1].content)
    
    return {
        "messages": [response["messages"][-1]],
        "agent_name": "github_agent_token",
        "company_summary": response["messages"][-1].content,
    }


In [10]:
# 테스트 케이스들
test_cases = [
    {
        "description": "테스트 1: 사용자 프로필 및 레포지토리 분석",
        "github_url": "https://github.com/Pseudo-Lab",
        "query": "Pseudo-Lab의 주요 레포지토리들을 분석하고, 어떤 기술 스택을 주로 사용하는지 알려줘",
    },
    {
        "description": "테스트 2: 특정 저장소 상세 분석",
        "github_url": "https://github.com/Pseudo-Lab",
        "query": "JobPT 레포지토리에 대해 상세히 요약해줘. README, 최근 커밋, 사용 언어, 기여자 정보를 포함해서",
    },
    {
        "description": "테스트 3: 개발 활동 분석",
        "github_url": "https://github.com/Pseudo-Lab",
        "query": "JobPT 프로젝트의 최근 개발 활동을 분석해줘. 최근 커밋들과 주요 기여자들을 중심으로",
    },
]

# 첫 번째 테스트 케이스로 State 생성
state = State(
    session_id=f"test_session_token_1",
    github_url=test_cases[0]['github_url'],
    company_name=test_cases[0]['github_url'].split('/')[-1],
    messages=[HumanMessage(content=test_cases[0]['query'])],
)

print("테스트 케이스 준비 완료!")
print(f"URL: {state.github_url}")
print(f"Query: {state.messages[0].content}")


테스트 케이스 준비 완료!
URL: https://github.com/Pseudo-Lab
Query: Pseudo-Lab의 주요 레포지토리들을 분석하고, 어떤 기술 스택을 주로 사용하는지 알려줘


In [11]:
# Agent 실행
result = await github_agent_token(state)


username:  Pseudo-Lab

Agent Response:
Pseudo-Lab의 주요 레포지토리들을 분석한 결과는 다음과 같습니다.

1. **DevFactory**
   - **설명**: Devfactory의 프로젝트 및 튜토리얼 모음 Repository
   - **주요 언어**: HTML, CSS, Python, JavaScript, TypeScript
   - **스타 수**: 64
   - **포크 수**: 3
   - **링크**: [DevFactory](https://github.com/Pseudo-Lab/DevFactory)

2. **Query-VendingMachine**
   - **설명**: Text2Sql 입문을 위한 레포지토리
   - **주요 언어**: Python
   - **스타 수**: 4
   - **포크 수**: 4
   - **링크**: [Query-VendingMachine](https://github.com/Pseudo-Lab/Query-VendingMachine)

3. **Hugging-Face-KREW-blog-explorer**
   - **설명**: Hugging Face 세계를 탐구하고 알리는 블로그 탐험가
   - **주요 언어**: 사용된 언어 정보 없음
   - **스타 수**: 8
   - **포크 수**: 6
   - **링크**: [Hugging-Face-KREW-blog-explorer](https://github.com/Pseudo-Lab/Hugging-Face-KREW-blog-explorer)

4. **Beyond-Why**
   - **설명**: Why, 그 너머를 탐구하고 실제 현실에서 활용할 수 있는 방법을 연구합니다.
   - **주요 언어**: Jupyter Notebook
   - **스타 수**: 7
   - **포크 수**: 5
   - **링크**: [Beyond-Why](https://github.com/Pseudo-Lab/Beyond-Why)

5. **Tut

In [12]:
# 결과 확인
print("\n" + "="*60)
print("최종 분석 결과:")
print("="*60)
print(result['company_summary'])



최종 분석 결과:
Pseudo-Lab의 주요 레포지토리들을 분석한 결과는 다음과 같습니다.

1. **DevFactory**
   - **설명**: Devfactory의 프로젝트 및 튜토리얼 모음 Repository
   - **주요 언어**: HTML, CSS, Python, JavaScript, TypeScript
   - **스타 수**: 64
   - **포크 수**: 3
   - **링크**: [DevFactory](https://github.com/Pseudo-Lab/DevFactory)

2. **Query-VendingMachine**
   - **설명**: Text2Sql 입문을 위한 레포지토리
   - **주요 언어**: Python
   - **스타 수**: 4
   - **포크 수**: 4
   - **링크**: [Query-VendingMachine](https://github.com/Pseudo-Lab/Query-VendingMachine)

3. **Hugging-Face-KREW-blog-explorer**
   - **설명**: Hugging Face 세계를 탐구하고 알리는 블로그 탐험가
   - **주요 언어**: 사용된 언어 정보 없음
   - **스타 수**: 8
   - **포크 수**: 6
   - **링크**: [Hugging-Face-KREW-blog-explorer](https://github.com/Pseudo-Lab/Hugging-Face-KREW-blog-explorer)

4. **Beyond-Why**
   - **설명**: Why, 그 너머를 탐구하고 실제 현실에서 활용할 수 있는 방법을 연구합니다.
   - **주요 언어**: Jupyter Notebook
   - **스타 수**: 7
   - **포크 수**: 5
   - **링크**: [Beyond-Why](https://github.com/Pseudo-Lab/Beyond-Why)

5. **Tutorial-Book**
   - **설명**: De

In [13]:
# 테스트 케이스 2 실행
state2 = State(
    session_id=f"test_session_token_2",
    github_url=test_cases[1]['github_url'],
    company_name=test_cases[1]['github_url'].split('/')[-1],
    messages=[HumanMessage(content=test_cases[1]['query'])],
)

print(f"URL: {state2.github_url}")
print(f"Query: {state2.messages[0].content}")
result2 = await github_agent_token(state2)


URL: https://github.com/Pseudo-Lab
Query: JobPT 레포지토리에 대해 상세히 요약해줘. README, 최근 커밋, 사용 언어, 기여자 정보를 포함해서
username:  Pseudo-Lab

Agent Response:
### JobPT 레포지토리 분석

**레포지토리 개요**
- **이름**: JobPT
- **설명**: 구직자의 이력서와 채용 공고를 매칭해주고, 채용 공고에 맞는 이력서를 검색하여 추천해주는 채용공고 매칭 시스템
- **언어**: Jupyter Notebook, JavaScript, Python, TypeScript, CSS, HTML 등
- **스타 수**: 33
- **포크 수**: 5
- **업데이트 날짜**: 2025년 11월 6일
- **링크**: [JobPT GitHub](https://github.com/Pseudo-Lab/JobPT)

---

**README 내용 요약**
- JobPT는 LLM 기반의 채용 공고 매칭 시스템으로, 구직자와 채용 공고 간의 매칭을 최적화하는 기능을 제공합니다.
- 주요 기능:
  1. LLM 기반의 이력서 매칭
  2. 채용 공고에 맞는 이력서 추천
  3. AI Agent를 통한 정보 제공 및 매칭
- 시스템 아키텍처와 흐름도도 포함되어 있으며, 다양한 API와 데이터베이스를 활용하여 기능을 구현하고 있습니다.

---

**최근 커밋**
1. [Merge pull request #138](https://github.com/Pseudo-Lab/JobPT/commit/04156117b26364ab8993808f9274da9815ddece9) - Dev merge (2025-11-06)
2. [Merge pull request #140](https://github.com/Pseudo-Lab/JobPT/commit/f3e591cb48013606f0d075959375535481afe34b) - CI 변경 (2025-11-03)
3. [fix: port 설정](htt

In [14]:
# 테스트 케이스 3 실행
state3 = State(
    session_id=f"test_session_token_3",
    github_url=test_cases[2]['github_url'],
    company_name=test_cases[2]['github_url'].split('/')[-1],
    messages=[HumanMessage(content=test_cases[2]['query'])],
)

print(f"URL: {state3.github_url}")
print(f"Query: {state3.messages[0].content}")
result3 = await github_agent_token(state3)


URL: https://github.com/Pseudo-Lab
Query: JobPT 프로젝트의 최근 개발 활동을 분석해줘. 최근 커밋들과 주요 기여자들을 중심으로
username:  Pseudo-Lab

Agent Response:
**JobPT 프로젝트 최근 개발 활동 분석**

1. **최근 커밋 내역**
   - 최근 5개의 커밋은 다음과 같습니다:
     - **[Merge pull request #138](https://github.com/Pseudo-Lab/JobPT/commit/04156117b26364ab8993808f9274da9815ddece9)**: 2025년 11월 6일, Minah Kim이 작성한 커밋으로, dev 브랜치와의 병합을 포함합니다.
     - **[Merge pull request #140](https://github.com/Pseudo-Lab/JobPT/commit/f3e591cb48013606f0d075959375535481afe34b)**: 2025년 11월 3일, Minah Kim이 작성한 커밋으로, 배포 관련 컨테이너 및 라우터 이름을 환경 접두사로 통일하는 작업입니다.
     - **[fix: port 번호 수정](https://github.com/Pseudo-Lab/JobPT/commit/801d05b4f8c029530c98c1281b55885682f69069)**: 2025년 11월 3일, yonghee Kim이 작성한 커밋으로, 포트 번호를 수정하는 내용입니다.
     - **[Merge pull request #139](https://github.com/Pseudo-Lab/JobPT/commit/fd4e2adf4d9a8f5bc84d2fc68cc9525a156a58cc)**: 2025년 11월 2일, Minah Kim이 작성한 커밋으로, 요약 기능 및 데이터베이스 관련 기능을 추가하는 작업입니다.
     - **[ci(deploy): unify container and router naming](h

In [15]:
# 개별 Tool 테스트 (선택사항)
print("=" * 60)
print("개별 Tool 테스트")
print("=" * 60)

# 1. 사용자 정보 조회
user_info = get_user_info.invoke({"username": "Pseudo-Lab"})
print("\n[User Info]")
print(json.dumps(user_info, indent=2, ensure_ascii=False))

# 2. 레포지토리 목록 조회
repos = list_user_repos.invoke({"username": "Pseudo-Lab", "limit": 5})
print("\n[Repositories (Top 5)]")
print(json.dumps(repos[:2], indent=2, ensure_ascii=False))  # 처음 2개만 출력

# 3. 특정 레포지토리 상세 정보
repo_details = get_repo_details.invoke({"owner": "Pseudo-Lab", "repo": "JobPT"})
print("\n[Repository Details]")
print(json.dumps(repo_details, indent=2, ensure_ascii=False))

# 4. README 내용
readme = get_repo_readme.invoke({"owner": "Pseudo-Lab", "repo": "JobPT"})
print("\n[README Preview]")
print(readme.get('content', '')[:500] + "...")  # 처음 500자만 출력

# 5. 최근 커밋
commits = get_repo_commits.invoke({"owner": "Pseudo-Lab", "repo": "JobPT", "limit": 3})
print("\n[Recent Commits]")
print(json.dumps(commits, indent=2, ensure_ascii=False))

# 6. 사용 언어
languages = get_repo_languages.invoke({"owner": "Pseudo-Lab", "repo": "JobPT"})
print("\n[Languages]")
print(json.dumps(languages, indent=2, ensure_ascii=False))

# 7. 기여자
contributors = get_repo_contributors.invoke({"owner": "Pseudo-Lab", "repo": "JobPT", "limit": 5})
print("\n[Contributors]")
print(json.dumps(contributors, indent=2, ensure_ascii=False))


개별 Tool 테스트

[User Info]
{
  "login": "Pseudo-Lab",
  "name": "가짜연구소 (Pseudo Lab)",
  "bio": "Non-profit community driving the popularization of machine learning research",
  "company": null,
  "location": null,
  "email": null,
  "followers": 297,
  "following": 0,
  "public_repos": 131,
  "created_at": "2020-09-08T04:32:47Z"
}

[Repositories (Top 5)]
[
  {
    "name": "DevFactory",
    "full_name": "Pseudo-Lab/DevFactory",
    "description": "Devfactory의 프로젝트 및 튜토리얼 모음 Repository",
    "html_url": "https://github.com/Pseudo-Lab/DevFactory",
    "language": "HTML",
    "stargazers_count": 64,
    "forks_count": 3,
    "updated_at": "2025-11-10T10:25:34Z",
    "topics": []
  },
  {
    "name": "Query-VendingMachine",
    "full_name": "Pseudo-Lab/Query-VendingMachine",
    "description": "Text2Sql 입문을 위한 레포지토리",
    "html_url": "https://github.com/Pseudo-Lab/Query-VendingMachine",
    "language": "Python",
    "stargazers_count": 4,
    "forks_count": 4,
    "updated_at": "2025-11-09T12